### Soil Moisture in Portales, New Mexico

**Station Name:**  p038

**Location:**  Portales, New Mexico, USA

**Archive:** [EarthScope Consortium](http://www.earthscope.org)

**Ellipsoidal Coordinates:**

- Latitude: 34.14726 degrees

- Longitude: -103.40734 degrees

- Height: 1212.982 meters

[Station Page at EarthScope](https://www.unavco.org/instrumentation/networks/status/nota/overview/P038)

[Google Maps Link](https://www.google.com/maps/place/34%C2%B008'50.1%22N+103%C2%B024'26.4%22W/@34.14725,-103.4073333,17z/data=!3m1!4b1!4m5!3m4!1s0x0:0x64e449f205085274!8m2!3d34.14725!4d-103.4073333) 

<p align="center">
<img src="https://www.unavco.org/data/gps-gnss/lib/images/station_images/P038.jpg" width="500"/>
</P>

P038 was a PBO site. The data from 2017 will be analyzed here as a test case. We will start by analyzing the data 
using the normal reflector height (GNSS-IR) processing. Then we will use those results to run the soil moisture code.

First, start by importing everything we need to run GNSS-IR:

In [ ]:
from gnssrefl.vwc import vwc
from gnssrefl.quickPhase import quickphase
from gnssrefl.vwc_input import vwc_input
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import os

from pathlib import Path

from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)

# import gnssrefl functions

#### Step 1: GNSS-IR
Begin by generating the SNR files. Although typically PBO sites do not have L2C 
data in their low-rate RINEX files, UNAVCO is providing these data in the "special" archive section
so that people can test out this code.

In [ ]:
# Set station, year, doy, and doy_end as variables so they can be used throughout the rest of this example
station = "p038"
year = 2017
doy = 1
doy_end = 365

rinex2snr(station=station, year=year, doy=doy, doy_end=doy_end, archive="special")

If you want to remind yourself why the L2C data are superior to the L1 data recorded for this receiver, use <code>quickLook</code>. You will need to import it from gnssrefl.

The analysis parameters are set up with <code>make_gnssir_input</code>. We only need the L2C data, so have set the parameter accordingly.

In [ ]:
make_gnssir_input(station, l2c=True)

Now we run <code>gnssir</code>. This will be needed for estimate a priori reflector heights for the soil moisture code.

In [ ]:
gnssir(station, year, doy, doy_end=doy_end)

#### Step 2: Soil Moisture

[Please read the soil moisture user manual.](https://github.com/kristinemlarson/gnssrefl/blob/master/docs/README_vwc.md) It is very short and has a lot of tips that will save you time.

We need a list of satellite tracks to use:

In [ ]:
# remember you can see what parameters are accepted by running: vwc_input?

vwc_input(station, 2017)

This creates a file that is stored in $REFL_CODE/input/p038_phaseRH.txt

Now we estimate the phase for each satellite track on each day:


In [ ]:
quickphase(station, year, doy, doy_end=doy_end)

Finally, convert the phase to volumetric water content.

This will result in 4 plots:
1. Phase results in geographic coordinates
2. Daily average phase
3. Model inputs
4. Final Results

These will all be saved in $REFL_CODE/Files

In [ ]:
vwc(station, year)

Thank you to Naoya Kadota for test driving this use case.